In [1]:
!git clone https://github.com/sooo66/semeval2026-task12-dataset.git

Cloning into 'semeval2026-task12-dataset'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 69 (delta 30), reused 51 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 6.72 MiB | 21.97 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [2]:
import json
import re

def clean_text(text):
    """
    پاک‌سازی اولیه متون
    """
    if not text or not isinstance(text, str):
        return ""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def load_and_merge_data(questions_path, docs_path):
    # ۱. خواندن فایل اسناد (docs.json)
    with open(docs_path, 'r', encoding='utf-8') as f:
        docs_raw = json.load(f)
        
    # ۲. تبدیل هوشمند لیست به دیکشنری برای جستجوی سریع بر اساس topic_id
    docs_dict = {}
    
    # بررسی می‌کنیم که آیا فایل اصلی لیستی از آیتم‌هاست
    if isinstance(docs_raw, list):
        for item in docs_raw:
            if not isinstance(item, dict):
                continue
                
            t_id = item.get("topic_id")
            if not t_id:
                continue
                
            if t_id not in docs_dict:
                docs_dict[t_id] = []
                
            # حالت اول: اگر آیتم شامل لیستی از اسناد در کلید docs باشد
            if "docs" in item or "documents" in item:
                inner_docs = item.get("docs", item.get("documents", []))
                for d in inner_docs:
                    if isinstance(d, dict):
                        # استخراج متن از دیکشنری داخلی
                        text_val = d.get("text", d.get("content", d.get("doc", "")))
                        docs_dict[t_id].append(text_val)
                    elif isinstance(d, str):
                        docs_dict[t_id].append(d)
            # حالت دوم: اگر خود آیتم مستقیماً یک سند باشد
            else:
                text_val = item.get("text", item.get("content", item.get("doc", "")))
                if text_val:
                    docs_dict[t_id].append(text_val)

    merged_dataset = []
    
    # ۳. خواندن فایل سوالات و اتصال به اسناد
    with open(questions_path, 'r', encoding='utf-8') as f:
        for line in f:
            if not line.strip():
                continue
                
            question_item = json.loads(line.strip())
            topic_id = question_item.get("topic_id")
            
            # استخراج اسناد از دیکشنری که بالاتر ساختیم (این بار بدون خطا)
            related_docs = docs_dict.get(topic_id, [])
            
            # پاک‌سازی و ترکیب متون
            cleaned_docs = [clean_text(doc) for doc in related_docs if doc]
            context_text = " ".join(cleaned_docs)
            
            # ۴. ساختار نهایی
            merged_item = {
                "id": question_item.get("id"),
                "topic_id": topic_id,
                "target_event": clean_text(question_item.get("target_event")),
                "options": {
                    "A": clean_text(question_item.get("option_A")),
                    "B": clean_text(question_item.get("option_B")),
                    "C": clean_text(question_item.get("option_C")),
                    "D": clean_text(question_item.get("option_D"))
                },
                "golden_answer": question_item.get("golden_answer"),
                "context": context_text 
            }
            merged_dataset.append(merged_item)
            
    return merged_dataset

# ==========================================
# اجرای کد
# ==========================================

QUESTIONS_FILE = 'semeval2026-task12-dataset/sample_data/questions.jsonl'
DOCS_FILE = 'semeval2026-task12-dataset/sample_data/docs.json'

dataset = load_and_merge_data(QUESTIONS_FILE, DOCS_FILE)

if dataset:
    print("✅ پردازش با موفقیت انجام شد!\n")
    print(f"🔹 رویداد هدف: {dataset[0]['target_event']}")
    print(f"🔹 گزینه‌ها:")
    for key, val in dataset[0]['options'].items():
        print(f"   {key}: {val}")
    print(f"🔹 پاسخ درست: {dataset[0]['golden_answer']}")
    print(f"🔹 طول متن زمینه (Context): {len(dataset[0]['context'])} کاراکتر")
else:
    print("مجموعه داده خالی است!")

✅ پردازش با موفقیت انجام شد!

🔹 رویداد هدف: Videos of the assassination circulated on social media.
🔹 گزینه‌ها:
   A: The shooter used a handmade gun.
   B: Security arrested the suspected gunman, Tetsuya Yamagami.
   C: Shinzo Abe became the deputy chief cabinet secretary in the early 2000s.
   D: A man fired twice at Shinzo Abe.
🔹 پاسخ درست: D
🔹 طول متن زمینه (Context): 94617 کاراکتر


In [3]:
# # در صورت نیاز کتابخانه‌های زیر را نصب کنید:
# # !pip install transformers torch

# import json

# def generate_prompt(sample, max_context_chars=4000):
#     """
#     این تابع پرامپت نهایی را برای ارسال به مدل زبانی می‌سازد.
#     برای جلوگیری از خطای محدودیت توکن در مدل پایه، طول کانتکست محدود شده است.
#     """
    
#     # برش متن زمینه (کانتکست) برای مدل پایه
#     context = sample['context']
#     if len(context) > max_context_chars:
#         context = context[:max_context_chars] + "\n... [متن به دلیل طولانی بودن برش داده شد] ..."

#     # طراحی ساختار پرامپت با دستورالعمل‌های سخت‌گیرانه (Strict Instructions)
#     prompt = f"""You are an expert AI system specialized in Abductive Event Reasoning. 
# Your task is to find the most likely direct cause of a given event based on the provided news context.

# Read the context carefully. Then, look at the Target Event and choose the most logical direct cause from the Options.

# Context:
# {context}

# Target Event: {sample['target_event']}

# Options:
# A) {sample['options']['A']}
# B) {sample['options']['B']}
# C) {sample['options']['C']}
# D) {sample['options']['D']}

# Which option is the most likely direct cause of the Target Event? 
# Output ONLY the letter corresponding to the correct option (A, B, C, or D). Do not write anything else.
# Answer: """
    
#     return prompt

# # ==========================================
# # تست ساخت پرامپت روی نمونه اول (dataset[0])
# # ==========================================

# sample_prompt = generate_prompt(dataset[0])
# print("✅ پرامپت طراحی شده برای ارسال به مدل:\n")
# print("-" * 50)
# print(sample_prompt)
# print("-" * 50)

# # ==========================================
# # شبیه‌سازی فراخوانی مدل (LLM Inference)
# # ==========================================
# # توجه: اگر در گوگل کولب هستید و منابع GPU دارید، می‌توانید کدهای زیر را از حالت کامنت خارج کنید 
# # تا یک مدل سبک را به صورت واقعی اجرا کنید.


# from transformers import pipeline

# print("\nدر حال بارگذاری مدل زبانی (این مرحله ممکن است کمی زمان‌بر باشد)...")
# # استفاده از یک مدل کوچک و سریع برای تست اولیه
# # برای نتایج بهتر در آینده می‌توانید از مدل‌های بزرگتر استفاده کنید
# pipe = pipeline("text-generation", model="Qwen/Qwen2.5-0.5B-Instruct", device_map="auto")

# print("مدل بارگذاری شد. در حال تولید پاسخ...")
# response = pipe(
#     sample_prompt, 
#     max_new_tokens=5, # ما فقط یک حرف می‌خواهیم، پس توکن‌های تولیدی را محدود می‌کنیم
#     temperature=0.1,  # دمای پایین برای پاسخ‌های قطعی‌تر و منطقی‌تر
#     do_sample=True
# )

# generated_text = response[0]['generated_text']
# # استخراج خروجی مدل (حذف پرامپت ورودی از خروجی نهایی)
# answer_only = generated_text.replace(sample_prompt, "").strip()

# print(f"\n🧠 خروجی مدل: {answer_only}")
# print(f"🎯 پاسخ درست (Golden Answer): {dataset[0]['golden_answer']}")


In [4]:
# # در صورت نیاز کتابخانه را نصب کنید:
# # !pip install scikit-learn

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import re

# def retrieve_relevant_context(sample, top_k=7):
#     """
#     این تابع با استفاده از TF-IDF مهم‌ترین جملات کانتکست را استخراج می‌کند.
#     top_k: تعداد جملاتی که می‌خواهیم استخراج کنیم.
#     """
#     context = sample['context']
    
#     # ۱. شکستن متن طولانی به جملات مجزا بر اساس نقطه، علامت سوال و تعجب
#     sentences = re.split(r'(?<=[.!?])\s+', context)
#     # فیلتر کردن جملات خیلی کوتاه یا خالی
#     sentences = [s.strip() for s in sentences if len(s.strip()) > 20]
    
#     if not sentences:
#         return ""

#     # ۲. ساخت کوئری: رویداد هدف + تمام گزینه‌ها
#     # دلیل: ما جملاتی را می‌خواهیم که هم درباره رویداد صحبت کنند و هم به علت‌های احتمالی اشاره داشته باشند
#     query = sample['target_event'] + " " + " ".join(sample['options'].values())

#     # ۳. برداری کردن متون با TF-IDF (و حذف کلمات توقف انگلیسی مانند the, is, at)
#     vectorizer = TfidfVectorizer(stop_words='english')
    
#     try:
#         # ساخت ماتریس جملات و بردار کوئری
#         tfidf_matrix = vectorizer.fit_transform(sentences)
#         query_vector = vectorizer.transform([query])
#     except ValueError:
#         # در صورتی که متن به شدت کوتاه باشد یا فقط شامل Stop-words باشد
#         return " ".join(sentences[:top_k])

#     # ۴. محاسبه شباهت کسینوسی بین کوئری و تک‌تک جملات
#     similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

#     # ۵. پیدا کردن ایندکس k جمله که بیشترین امتیاز شباهت را دارند
#     # argsort آرایه را صعودی مرتب می‌کند، پس k تای آخر را جدا می‌کنیم
#     top_indices = similarities.argsort()[-top_k:]

#     # ۶. نکته کلیدی: مرتب‌سازی مجدد ایندکس‌ها بر اساس ترتیب اصلی متن!
#     # این کار باعث می‌شود جملات استخراج شده، انسجام و سیر زمانی خود را حفظ کنند و مدل گیج نشود.
#     top_indices_sorted = sorted(top_indices)
    
#     # ترکیب جملات برتر
#     best_sentences = [sentences[i] for i in top_indices_sorted]
#     filtered_context = " ".join(best_sentences)

#     return filtered_context

# # ==========================================
# # تست سیستم بازیابی روی نمونه شینزو آبه (dataset[0])
# # ==========================================

# # فراخوانی تابع فیلتر هوشمند
# smart_context = retrieve_relevant_context(dataset[0], top_k=5)

# print("✅ کانتکست فیلتر شده (Smart Context):\n")
# print("-" * 50)
# print(smart_context)
# print("-" * 50)
# print(f"\nطول کانتکست جدید: {len(smart_context)} کاراکتر (کاهش چشمگیر از 94,617!)")

In [5]:
# در صورت نیاز کتابخانه‌های زیر را نصب کنید:
# !pip install transformers torch scikit-learn sentence-transformers

import json
import re
import torch
from collections import Counter
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ==========================================
# ۰. بارگذاری مدل Embedding قدرتمندتر
# ==========================================
print("در حال بارگذاری مدل Embedding قدرتمند (BGE-Large)...")
# تغییر مدل به bge-large برای درک عمیق‌تر مفاهیم سیاسی و خبری
embedder = SentenceTransformer('BAAI/bge-large-en-v1.5') 

# ==========================================
# ۱. تابع بازیابی اطلاعات (Semantic RAG)
# ==========================================
def retrieve_relevant_context(sample, top_k=6):
    context = sample['context']
    sentences = re.split(r'(?<=[.!?])\s+', context)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 20]
    
    if not sentences: return ""

    query = sample['target_event'] + " " + " ".join(sample['options'].values())
    sentence_embeddings = embedder.encode(sentences)
    query_embedding = embedder.encode([query])

    similarities = cosine_similarity(query_embedding, sentence_embeddings).flatten()
    top_indices = similarities.argsort()[-top_k:]
    top_indices_sorted = sorted(top_indices) 
    
    return " ".join([sentences[i] for i in top_indices_sorted])

# ==========================================
# ۲. تابع ساخت پیام‌ها با Few-Shot CoT
# ==========================================
def generate_messages(sample, filtered_context):
    """
    اضافه کردن مثال‌های آموزشی (Few-Shot) برای کالیبره کردن مدل
    """
    # مثال‌هایی از دیتاست آموزشی برای یادگیری مدل
    few_shot_examples = """
Example 1:
Context: The building's structural integrity was compromised after the explosion. Investigators found traces of C4.
Target Event: The roof collapsed.
Options: A) Heavy rain started. B) An explosive device detonated. C) The building was old. D) Firefighters arrived.
Reasoning: The context mentions an explosion and C4, which is an explosive. Detonation of such a device is the direct cause of the collapse.
<ANSWER>B</ANSWER>

Example 2:
Context: Multiple witness reports and CCTV footage showed two different individuals firing from the balcony and the street level.
Target Event: Chaos erupted in the plaza.
Options: A) A person fired from the balcony. B) A car honked. C) A person fired from the street. D) Police arrived later.
Reasoning: Both shooters (balcony and street) caused the chaos directly according to the footage.
<ANSWER>A,C</ANSWER>
"""

    messages = [
        {
            "role": "system", 
            "content": f"You are an expert AI specialized in Abductive Event Reasoning. Use the following examples as a guide:\n{few_shot_examples}\n\nTask: First, briefly reason. Then, output the uppercase letter(s) of the correct option(s) inside <ANSWER> tags."
        },
        {
            "role": "user", 
            "content": f"Context: {filtered_context}\n\nTarget Event: {sample['target_event']}\n\nOptions:\nA) {sample['options']['A']}\nB) {sample['options']['B']}\nC) {sample['options']['C']}\nD) {sample['options']['D']}\n\nReason and output final answer in <ANSWER> tags."
        }
    ]
    return messages

# ==========================================
# ۳. بارگذاری مدل زبانی اصلی
# ==========================================
print("\nدر حال بارگذاری مدل زبانی Qwen2.5-7B...")
pipe = pipeline(
    "text-generation", 
    model="Qwen/Qwen2.5-7B-Instruct", 
    device_map="auto",
    torch_dtype=torch.float16 
)
print("✅ آماده‌سازی نهایی انجام شد.")

در حال بارگذاری مدل Embedding قدرتمند (BGE-Large)...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ آماده‌سازی نهایی انجام شد.


In [6]:
import json
import re
from tqdm import tqdm
from collections import Counter

def calculate_score(predicted_answers_str, golden_answers_str):
    if predicted_answers_str == "نامشخص": return 0.0
    P = set([ans.strip() for ans in predicted_answers_str.split(',') if ans.strip()])
    G = set([ans.strip() for ans in golden_answers_str.split(',') if ans.strip()])
    if P == G and len(P) > 0: return 1.0  
    elif P.issubset(G) and len(P) > 0: return 0.5  
    else: return 0.0  

# ==========================================
# ارزیابی با سیستم Majority Voting (3 تکرار)
# ==========================================
print(f"\nشروع ارزیابی روی {len(dataset)} نمونه با سیستم Majority Voting...")
total_score = 0
results_log = []

for index, sample in enumerate(tqdm(dataset, desc="Evaluating")):
    smart_context = retrieve_relevant_context(sample, top_k=6)
    messages = generate_messages(sample, smart_context)
    formatted_prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # اجرای ۳ باره مدل برای هر سوال (Self-Consistency)
    votes = []
    for _ in range(3):
        response = pipe(
            formatted_prompt, 
            max_new_tokens=150, 
            temperature=0.4, # دمای بالاتر برای تنوع در استدلال
            do_sample=True,
            pad_token_id=pipe.tokenizer.eos_token_id
        )
        output = response[0]['generated_text'].replace(formatted_prompt, "").strip()
        match = re.search(r'<ANSWER>\s*([A-D](?:\s*,\s*[A-D])*)\s*</ANSWER>', output)
        if match:
            votes.append(match.group(1).replace(' ', ''))
        else:
            fallback = re.search(r'[A-D](?:\s*,\s*[A-D])*', output)
            votes.append(fallback.group(0).replace(' ', '') if fallback else "نامشخص")
    
    # انتخاب پرتکرارترین جواب (Majority Vote)
    final_prediction = Counter(votes).most_common(1)[0][0]
    
    score = calculate_score(final_prediction, sample['golden_answer'])
    total_score += score
    results_log.append({"id": sample['id'], "predicted": final_prediction, "golden": sample['golden_answer'], "score": score})

average_score = total_score / len(dataset)
print(f"\n🏆 دقت نهایی با Self-Consistency: {average_score * 100:.2f}%")


شروع ارزیابی روی 200 نمونه با سیستم Majority Voting...


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]Passing `generation_config` together with generation-related arguments=({'do_sample', 'temperature', 'max_new_tokens', 'pad_token_id'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more in


🏆 دقت نهایی با Self-Consistency: 56.50%
